In [1]:
pip install requests

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pytz

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install matplotlib


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install python-telegram-bot

     |████████████████████████████████| 549 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 3.8 MB/s eta 0:00:011
     |████████████████████████████████| 74 kB 17.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install keys

ERROR: Could not find a version that satisfies the requirement keys (from versions: none)
ERROR: No matching distribution found for keys
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
from selenium.webdriver.common.keys import Keys

In [13]:
import numpy as np
import argparse
import pickle
import cv2
import os
import time
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import requests
from datetime import datetime
import pytz
import folium
from geopy.geocoders import Nominatim
from selenium import webdriver
from telegram.ext import Updater,filters, MessageHandler
from telegram.ext import *
from selenium.webdriver.common.keys import Keys


# Load the violence detection model
model = load_model('/Users/faizahkureshi/Desktop/GAIP Project/modelnew.h5')
async def start_commmand(update, context):
    await update.message.reply_text('Hello! Welcome To Store!')
# Initialize the video processing variables
Q = deque(maxlen=128)
vs = None
(W, H) = (None, None)
writer = None
count = 0
counter = 0

# Telegram Bot API token
telegram_auth_token = "6336511764:AAG3aM1bJ6wXQ_7EG8ZgpLucpmZmGCxINX8"
telegram_group_chat_id = "-1001926862448"
application = Application.builder().token(Keys.token).build()
# Commands
application.add_handler(CommandHandler('start', start_commmand))

# Run bot
application.run_polling(1.0)
# Initialize the Telegram bot updater
#updater = Updater(telegram_auth_token,True)
#dispatcher = updater.dispatcher

# Function to process video messages
def process_video(update, context):
    global vs, W, H, writer, counter

    # Get the video file from the update
    video = context.bot.get_file(update.message.video.file_id)
    video_path = video.download()

    # Open the video file
    vs = cv2.VideoCapture(video_path)

    while True:
        # Read the next frame from the video
        ret, frame = vs.read()

        if not ret:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(128, 128, 3) / 255

        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)

        results = np.array(Q).mean(axis=0)
        i = (preds > 0.50)[0]
        label = i
        print(i)
        text_color = (0, 255, 0)  # default: green

        if label:  # Violence prob
            text_color = (0, 0, 255)  # red

        if label:
            print("1")
            text = "Violence: {}".format(label)
            FONT = cv2.FONT_HERSHEY_SIMPLEX

            cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)

            if writer is None:
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter("output/v_output.avi", fourcc, 30, (W, H), True)

            writer.write(output)

            image_path = f"output/output_{counter}.png"
            plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
            plt.savefig(image_path)
            plt.close()

            message = f"Alert! Violence detected at {datetime.now().strftime('%d-%m-%Y %H:%M:%S')}"
            send_message_to_group(message)

            send_image("Check out this image!", photo_path=image_path)

            counter += 1

    print("[INFO] Cleaning up...")
    writer.release()
    vs.release()

def send_message_to_group(message):
    telegram_api_url = f"https://api.telegram.org/bot{telegram_auth_token}/sendMessage"
    params = {
        "chat_id": telegram_group_chat_id,
        "text": message
    }
    response = requests.get(telegram_api_url, params=params)
    if response.status_code == 200:
        print("Message sent successfully!")
    else:
        print("Failed to send the message.")

def send_image(message, photo_path=None):
    telegram_api_url = f"https://api.telegram.org/bot{telegram_auth_token}/sendPhoto"

    params = {
        "chat_id": telegram_group_chat_id,
        "caption": message
    }

    files = None
    if photo_path:
        files = {"photo": open(photo_path, "rb")}

    response = requests.post(telegram_api_url, params=params, files=files)
    if response.status_code == 200:
        print("Image sent successfully!")
    else:
        print("Failed to send the image.")



# Start the Telegram bot
updater.start_polling()

# Handler to process video messages
video_handler = MessageHandler(filters.VIDEO, process_video)
dispatcher.add_handler(video_handler)

# Keep the program running until interrupted
while True:
    time.sleep(1)


AttributeError: type object 'Keys' has no attribute 'token'